In [3]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import optuna

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Create DataFrame for visualization
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df['species'] = df['target'].map({i: target_names[i] for i in range(len(target_names))})

print("Iris Dataset Overview:")
print(df.head())
print("\nTarget distribution:")
print(df['species'].value_counts())

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the objective function for Optuna
def objective(trial):
    model_name = trial.suggest_categorical("model", ["RandomForest", "SVC"])
    
    if model_name == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 50, 300)
        max_depth = trial.suggest_int("max_depth", 2, 20)
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
    
    elif model_name == "SVC":
        C = trial.suggest_float("C", 1e-3, 1e2, log=True)
        gamma = trial.suggest_float("gamma", 1e-4, 1e-1, log=True)
        model = SVC(
            C=C,
            gamma=gamma,
            random_state=42
        )
    
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

# Create and run the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Print optimization results
print("\nOptimization Results:")
print("Best parameters:", study.best_params)
print("Best accuracy:", study.best_value)

# Train final model with best parameters
best_params = study.best_params.copy()
model_type = best_params.pop("model")

if model_type == "RandomForest":
    final_model = RandomForestClassifier(**best_params, random_state=42)
elif model_type == "SVC":
    final_model = SVC(**best_params, random_state=42)

final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

# Evaluate final model
print("\nFinal Model Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

# Feature importance (if RandomForest was selected)
if model_type == "RandomForest":
    print("\nFeature Importances:")
    importances = final_model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"{name}: {importance:.4f}")

[I 2025-05-11 12:11:04,988] A new study created in memory with name: no-name-416e7542-6b77-4e9f-b7f9-55d94bc8e298


Iris Dataset Overview:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target species  
0       0  setosa  
1       0  setosa  
2       0  setosa  
3       0  setosa  
4       0  setosa  

Target distribution:
species
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-05-11 12:11:05,022] Trial 0 finished with value: 0.9666666666666667 and parameters: {'model': 'SVC', 'C': 5.491526926790928, 'gamma': 0.014566655340951022}. Best is trial 0 with value: 0.9666666666666667.
[I 2025-05-11 12:11:05,030] Trial 1 finished with value: 0.8333333333333334 and parameters: {'model': 'SVC', 'C': 0.01659344774035328, 'gamma': 0.00018488460106933722}. Best is trial 0 with value: 0.9666666666666667.
[I 2025-05-11 12:11:05,051] Trial 2 finished with value: 0.9333333333333333 and parameters: {'model': 'SVC', 'C': 5.9843012756422125, 'gamma': 0.005655269845391296}. Best is trial 0 with value: 0.9666666666666667.
[I 2025-05-11 12:11:05,066] Trial 3 finished with value: 1.0 and parameters: {'model': 'SVC', 'C': 12.429983353230117, 'gamma': 0.0400831219698813}. Best is trial 3 with value: 1.0.
[I 2025-05-11 12:11:05,076] Trial 4 finished with value: 0.8333333333333334 and parameters: {'model': 'SVC', 'C': 16.741932691787945, 'gamma': 0.00014154097148201918}. Best i